# Data Audit – Notebook

**Uwaga**  
Ten notatnik pełni rolę *„brudnopisu”* – pokazuje proces audytu danych krok po kroku.  
Kod i wykresy w tym notebooku mają charakter roboczy i nie są dopracowane pod względem estetycznym.  

Estetyczne i finalne wersje wyników znajdują się w:  
- `scripts/data_audit.py` - czysty, uporządkowany kod,  
- `reports/audit_full.txt` oraz `reports/audit_extended.txt` - finalne raporty tekstowe z audytu.  

Celem tego notatnika jest **pokazanie procesu sprawdzania jakości danych**, w tym:  
- testy spójności i integralności między tabelami,  
- sprawdzanie unikalności kluczy głównych,  
- sprawdzanie wartości (np. brak NULL),  
- wykrywanie anomalii.

## Importy i konfiguracjja środowiska

W tej komórce importujemy niezbędne biblioteki:

- `os` - do operacji na systemie plików i zmiennych środowiskowych,
- `pandas` - do pracy z danymi w formie DataFrame,
- `dotenv` - do wczytywania zmiennych środowiskowych z pliku `.env`,
- `sqlalchemy` - do łączenia się z bazą danych i wykonywania zapytań SQL.

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

## Wczytywanie zmiennych środowiskowych

Wczytujemy dane do logowania do bazy z pliku `.env`, żeby nie trzymać haseł bezpośrednio w kodzie.  

Zmienne w `.env`:

- `DB_HOST` - adres serwera bazy,
- `DB_NAME` - nazwa bazy danych,
- `DB_USER` - użytkownik bazy,
- `DB_PASS` - hasło użytkownika.

In [ ]:
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")

## Połączenie z bazą danych

Tworzymy połączenie do bazy danych PostgreSQL przy użyciu `SQLAlchemy`.  
Dzięki temu możemy wykonywać zapytania SQL bezpośrednio w Pythonie.

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

with engine.connect() as conn:
    version = conn.execute(text("SELECT version()")).scalar()
    print(f"Połączono z bazą. Wersja PostgreSQL: {version}")

## Lista tabel w schemacie `public`

W tej części pobieramy wszystkie tabele ze schematu `public`, aby wiedzieć które tabele trzeba poddać audytowi.

In [ ]:
tables = pd.read_sql(
    "SELECT table_name FROM information_schema.tables WHERE table_schema='public'",
    engine
)["table_name"].tolist()

print(f"\nZnaleziono {len(tables)} tabel: {tables}")

## Folder na raporty audytu

Tworzymy folder `data/audit_reports`, gdzie będą zapisywane pliki CSV z wynikami audytu.  
Funkcja `os.makedirs(..., exist_ok=True)` zapewnia, że folder zostanie utworzony tylko jeśli jeszcze nie istnieje.

In [ ]:
out_dir = "data/audit_reports"
os.makedirs(out_dir, exist_ok=True)

## Audyt tabel

Dla każdej tabeli wykonujemy:

1. Podgląd 5 pierwszych wierszy (`preview`),
2. Zliczenie wszystkich wierszy (`row_count`),
3. Sprawdzenie braków danych (`NULL`) w podglądzie,
4. Sprawdzenie unikalności wartości i nietypowych znaków (np. średników) w danych,
5. Zapis wyników do zbiorczego raportu.

Każda tabela jest analizowana osobno, a wyniki są zbierane w liście `all_tables_report`.

In [2]:
all_tables_report = []

for table in tables:
    print(f"\n=== Audyt tabeli: {table} ===")
    preview = pd.read_sql(f'SELECT * FROM "{table}" LIMIT 5', engine)
    print(preview)
    
    row_count = pd.read_sql(f'SELECT COUNT(*) FROM "{table}"', engine).iloc[0, 0]
    print(f"Liczba wierszy: {row_count}")
    
    null_counts = preview.isnull().sum()
    print("\nBraki danych (NULL):")
    print(null_counts)
    
    uwagi = []
    for col in preview.columns:
        if preview[col].nunique(dropna=True) == len(preview):
            uwagi.append(f"Kolumna '{col}' ma unikalne wartości w podglądzie")
        if any(";" in str(v) for v in preview[col].dropna()):
            uwagi.append(f"Kolumna '{col}' zawiera średnik w danych")
    
    all_tables_report.append({
        "tabela": table,
        "wiersze": row_count,
        "braki_NULL": null_counts.to_dict(),
        "uwagi": "; ".join(uwagi) if uwagi else ""
    })

NameError: name 'tables' is not defined

## Zapis raportu zbiorczego

Tworzymy dataframe z listy raportów, a następnie zapisujemy go jako CSV w folderze `data/audit_reports` pod nazwą `audit_summary.csv`.  

Plik zawiera:

- nazwę tabeli,
- liczbę wierszy,
- braki danych (`NULL`) w podglądzie,
- uwagi dotyczące unikalności kolumn i innych anomalii w danych.

In [ ]:
report_df = pd.DataFrame(all_tables_report)
report_path = os.path.join(out_dir, "audit_summary.csv")
report_df.to_csv(report_path, index=False, encoding="utf-8-sig")

print(f"\nAudyt zakończony. Raport zapisany do: {report_path}")

## Podsumowanie

Po wykonaniu audytu mamy pełny raport wszystkich tabel, który możemy wykorzystać do dalszego czyszczenia i analiz danych.